In [1]:
# l'importation des librairies
import numpy as np
import random
import math

class EnvGrid:
    def __init__(self):
        # Création de l'environnement avec des valeurs initiales à 0
        self.grid = np.zeros((10, 10))
        # Valeur du but
        self.grid[-1, -1] = 10
        # Valeurs des obstacles
        self.grid[0, 5] = 1
        self.grid[0, 6] = 1
        self.grid[3, 3] = 1
        self.grid[3, 4] = 1
        self.grid[6, 5] = 1
        self.grid[6, 6] = 1
        self.y = 0
        self.x = 0
        # Liste des actions possibles : bas, droite, gauche,  haut
        self.actions = [(1, 0), (0, 1), (0, -1), (-1,0)]
        # Matrice de récompenses initialisée à 0
        self.R = np.zeros((10, 10))

    # Réinitialisation de la position à (0,0)
    def reset(self):
        self.i = 0
        self.j = 0
        # Renvoie la position sous forme d'un nombre (entier)
        return self.i * 10 + self.j


    def step(self, action):
        # Récupère l'action choisie
        di, dj = self.actions[action]
        # Met à jour la position en fonction de l'action et de l'environnement (pour ne pas sortir de la grille)
        self.i = max(0, min(self.i + di, 9))
        self.j = max(0, min(self.j + dj, 9))
        # Si la nouvelle position est un obstacle, la récompense est négative
        if self.grid[self.i, self.j] == 1:
            self.R[self.i, self.j] = -10
            return self.i * 10 + self.j, self.i, self.j
        # Si la nouvelle position est le but, la récompense est positive de valeur 10
        elif self.i == 9 and self.j == 9:
            self.R[self.i, self.j] = 10
            return self.i * 10 + self.j, self.i, self.j
        # Sinon, la récompense est basée sur la distance à la position finale
        else:
            goal = (9, 9)
            dist = 1/math.sqrt(pow((goal[0] - self.i),2)+pow((goal[1] - self.j),2))
            self.R[self.i, self.j] = dist
            return self.i * 10 + self.j, self.i, self.j


    def is_finished(self):
        # Renvoie True si la position actuelle est le but, False sinon
        return self.grid[self.i, self.j] == 10

def take_action():
    # Action aléatoire
    action = random.randint(0, 3)
    return action

def q_learning(env, gamma=0.9, n_iter=1000):
    # Matrice Q initialisée à 0
    Q = np.zeros((100, 4))
    # Boucle sur le nombre d'itérations
    for _ in range(n_iter):
        # Réinitialisation de l'environnement
        st = env.reset()
        # Boucle tant que la position actuelle n'est pas le but
        while not env.is_finished():
            # Choix de l'action
            at = take_action()
            # Mise à jour de l'environnement en fonction de l'action choisie
            stp1, t , z = env.step(at)
            # Mise à jour de la matrice Q selon la formule de Q-learning
            Q[st, at] = env.R[t, z] + gamma * Q[st,np.argmax(Q[st])] 
            # Mise à jour de la position actuelle
            st = stp1
    return Q

def generate_shortest_path(env, Q):
    # Chemin initialisé avec la position de départ
    path = [(0,0),]
    st = env.reset()
    # Boucle tant que la position actuelle n'est pas le but
    while not env.is_finished():
        # Choix de l'action qui maximise Q
        at = np.argmax(Q[st])
        # Mise à jour du position du robot en fonction de l'action choisie
        st, i , j = env.step(at)
        # Ajout de la nouvelle position au chemin
        path.append((i, j))
    return path

In [2]:
env = EnvGrid()
# Apprentissage de Q
Q = q_learning(env)
# Génération du chemin le plus court selon Q
path = generate_shortest_path(env, Q)
# Affichage du chemin
print(path)

[(0, 0), (1, 0), (1, 1), (2, 1), (2, 2), (3, 2), (4, 2), (4, 3), (4, 4), (5, 4), (5, 5), (5, 6), (5, 7), (6, 7), (7, 7), (8, 7), (8, 8), (9, 8), (9, 9)]
